In [26]:
import pandas as pd
import sqlalchemy as sal
from sqlalchemy import create_engine

In [27]:
df = pd.read_csv('FMCG_2022_2024.csv')

In [15]:
df.head()

,date,sku,brand,segment,category,channel,region,pack_type,price_unit,promotion_flag,delivery_days,stock_available,delivered_qty,units_sold
0,2022-01-21,MI-006,MiBrand1,Milk-Seg3,Milk,Retail,PL-Central,Multipack,2.38,0,1,141,128,9
1,2022-01-21,MI-006,MiBrand1,Milk-Seg3,Milk,Retail,PL-North,Single,1.55,1,3,0,129,0
2,2022-01-21,MI-006,MiBrand1,Milk-Seg3,Milk,Retail,PL-South,Carton,4.00,0,5,118,161,8
3,2022-01-21,MI-006,MiBrand1,Milk-Seg3,Milk,Discount,PL-Central,Single,5.16,0,2,81,114,7
4,2022-01-21,MI-006,MiBrand1,Milk-Seg3,Milk,Discount,PL-North,Single,7.66,0,4,148,204,12


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190757 entries, 0 to 190756
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   date             190757 non-null  object 
 1   sku              190757 non-null  object 
 2   brand            190757 non-null  object 
 3   segment          190757 non-null  object 
 4   category         190757 non-null  object 
 5   channel          190757 non-null  object 
 6   region           190757 non-null  object 
 7   pack_type        190757 non-null  object 
 8   price_unit       190757 non-null  float64
 9   promotion_flag   190757 non-null  int64  
 10  delivery_days    190757 non-null  int64  
 11  stock_available  190757 non-null  int64  
 12  delivered_qty    190757 non-null  int64  
 13  units_sold       190757 non-null  int64  
dtypes: float64(1), int64(5), object(8)
memory usage: 20.4+ MB


In [17]:
df.columns

Index(['date', 'sku', 'brand', 'segment', 'category', 'channel', 'region',
       'pack_type', 'price_unit', 'promotion_flag', 'delivery_days',
       'stock_available', 'delivered_qty', 'units_sold'],
      dtype='object')

# Data Cleaning & Preparation

In [18]:
df['date'] = pd.to_datetime(df['date'])

In [19]:
df.isna().sum()

date               0
sku                0
brand              0
segment            0
category           0
channel            0
region             0
pack_type          0
price_unit         0
promotion_flag     0
delivery_days      0
stock_available    0
delivered_qty      0
units_sold         0
dtype: int64

In [20]:
for col in ['brand','segment','category','channel','region','pack_type']:
    print(col, df[col].nunique(), df[col].unique()[:5])

brand 14 ['MiBrand1' 'MiBrand4' 'YoBrand4' 'YoBrand2' 'YoBrand1']
segment 13 ['Milk-Seg3' 'Milk-Seg2' 'Yogurt-Seg2' 'Yogurt-Seg1' 'Yogurt-Seg3']
category 5 ['Milk' 'Yogurt' 'ReadyMeal' 'Juice' 'SnackBar']
channel 3 ['Retail' 'Discount' 'E-commerce']
region 3 ['PL-Central' 'PL-North' 'PL-South']
pack_type 3 ['Multipack' 'Single' 'Carton']


# Feature Engineering

In [21]:
df['revenue'] = df['price_unit'] * df['units_sold']
df['promotion_flag'] = df['promotion_flag'].map({0:'No',1:'Yes'})

In [22]:
df[df['units_sold'] > df['delivered_qty']]
df[df['price_unit'] <= 0]

,date,sku,brand,segment,category,channel,region,pack_type,price_unit,promotion_flag,delivery_days,stock_available,delivered_qty,units_sold,revenue


# Load the data into sql server using replace option

In [29]:
engine = sal.create_engine(
    "mssql+pyodbc://@DESKTOP-RAQ39CC\\SQLEXPRESS/fmcg_sales?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
)
conn = engine.connect()

In [30]:
# simpan dataframe ke tabel bernama "daily_sales" di schema dbo
df.to_sql(
    'daily_sales',    # nama tabel di SQL Server
    con=conn,
    schema='dbo',     # schema default biasanya dbo
    index=False,
    if_exists='replace'   # replace (buat ulang), atau append (tambah data)
)

37